# Video analysis
## Meanshift and Camshift
### Meanshift
Meanshift decides the point such that the circle centered at that point has maximum pixel distribution. Iteratively the circle is moved until　the distance between the centroid of distribution and the center of the circle is smaller than desired error. Usually, Histogram backprojected image and the initial target position is specified.

### Meanshift in OpenCV

In [1]:
import numpy as np
import cv2

In [16]:
cap = cv2.VideoCapture('inputs/slow.flv')

# take the first frame of the video
ret, frame = cap.read()

# initial window
col, row, width, height = 300, 200, 100, 50
track_window = (col, row, width, height)

# set up the ROI for tracking
roi = frame[row:row+height, col:col+width]
hsv_roi = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60., 32.)), np.array((180., 255., 255.)))
roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

# set up the termination criteria, either 10 iteration or move by at least 1 pt
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while(1):
    ret, frame = cap.read()
    
    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
        
        ret, track_window = cv2.meanShift(dst, track_window, term_crit)
        col, row, width, height = track_window
        
        img2 = cv2.rectangle(frame, (col, row), (col+width, row+height), 255, 2)
        cv2.startWindowThread()  # This line is not needed when you execute from terminal.
        cv2.imshow('img2', img2)
        
        key = cv2.waitKey(60) & 0xff
        if key == 27:
            break
        else:
            cv2.imwrite("inputs/" + chr(key) + ".jpg", img2)

    else:
        break

cv2.waitKey(1)  # This line is not needed when you execute from terminal.
cv2.destroyAllWindows()
cv2.waitKey(1)  # This line is not needed when you execute from terminal.
cap.release()

`cv2.meanShift()` has 3 arguments, back projection of the object histogram, initial search window, and criteria.

### Camshift
Above method has one problem that the window always has the same size whether the car is very far or very close to the camera. Window size and rotation should be adapted to the target.  
Camshift resolves this problem. Once meanshift is applied, it updates the size of the window as $s = 2 \times \sqrt{\frac{M_{00}}{256}}$. It also calculates the orientation of the best fitting ellipse to it.

In [12]:
cap = cv2.VideoCapture('inputs/slow.flv')

# take first frame of the video
ret, frame = cap.read()

# setup initial location of window
col, row, width, height = 300, 200, 100, 50
track_window = (col, row, width, height)

# set up the ROI for tracking
roi = frame[row:row+height, col:col+width]
hsv_roi = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60., 32.)), np.array((180., 255., 255.)))
roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

# set up the termination criteria, either 10 iteration or move by at least 1 pt
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while(1):
    ret, frame = cap.read()
    
    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
        
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)
        
        # draw it on image
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        
        img2 = cv2.polylines(frame, [pts], True, 255, 2)
        cv2.startWindowThread()  # This line is not needed when you execute from terminal.
        cv2.imshow('img2', img2)
        
        key = cv2.waitKey(60) & 0xff
        if key == 27:
            break
        else:
            cv2.imwrite('inputs/' + chr(key) + ".jpg", img2)
    
    else:
        break
        
cv2.waitKey(1)  # This line is not needed when you execute from terminal.
cv2.destroyAllWindows()
cv2.waitKey(1)  # This line is not needed when you execute from terminal.
cap.release()

## Optical flow
Optical flow is 2D vector field where each vector is a displacement vector showing the movement of points from first frame to second.  
Optical flow has many applications like structure from motion, video compression, video stabilization etc.  
Optical flow works on several assumptions:
1. The pixel intensities of an object do not change between consecutive frmaes.
2. Neighbouring pixels have similar motion.

Following equation is obtained:
$$
I(x, y, t) = I(x+dx, y+dy, t+dt)
$$
It is changed as:
$$
f_xu + f_yv + f_t = 0
$$

### Lucas-Kanade method
Above equation can't be solved because one equation against two unknown variables $u, v$.  
Lucas-Kanade method assumes that a 3 x 3 patch around the point have the same motion, and get 9 equations. A better solution is obtained with least square fit method.

### Lucas-Kanade optical flow in OpenCV
`cv2.calcOpticalFlowPyrLK(prevImg, nextImg, prevPts, nextPts,)` returns (nextPts, status, err).

In [22]:
cap = cv2.VideoCapture('inputs/slow.flv')

# params for ShiTomasi corner detection
feature_params = dict(maxCorners = 100, qualityLevel = 0.3, minDistance = 7, blockSize = 7)

# params for lucas kanade optical flow
lk_params = dict(winSize = (15, 15), maxLevel = 2, 
                               criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# create some random colors
color = np.random.randint(0 ,255, (100, 3))

# take the first frame and find corner in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

# create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

while(1):
    ret, frame = cap.read()
    if not ret:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    
    # select good points
    if p1 is not None:
        good_new = p1[st==1]
        good_old = p0[st==1]

    # draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        a, b, c, d = map(int, (a, b, c, d))
        mask = cv2.line(mask, (a, b), (c, d), color[i].tolist(), 2)
        frame = cv2.circle(frame, (a, b), 5, color[i].tolist(), -1)
    img = cv2.add(frame, mask)
    
    cv2.startWindowThread()  # This line is not needed when you execute from terminal.
    cv2.imshow('frame', img)
    key = cv2.waitKey(30) & 0xff
    if key == 27:
        break
        
    # now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

cap.release()
cv2.waitKey(1)  # This line is not needed when you execute from terminal.
cv2.destroyAllWindows()
cv2.waitKey(1)  # This line is not needed when you execute from terminal.


-1

### Dense optical flow in OpenCV
`cv2.calcOpticalFlowFarneback(prevImg, nextImg, flow, pyr_scale, levels, winsize, iterations, poly_n, poly_sigma, flags)` computes the optical flow for all the points in the frame while above method computes for corner. It returns 2-channel array with optical flow vectors.

In [25]:
cap = cv2.VideoCapture('inputs/vtest.avi')

ret, frame1 = cap.read()
prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255

while(1):
    ret, frame2 = cap.read()
    if not ret:
        break
    next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    
    flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang * 180 / np.pi / 2
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    
    cv2.startWindowThread()  # This line is not needed when you execute from terminal.
    cv2.imshow('frame2', bgr)
    key = cv2.waitKey(30) & 0xff
    
    if key == 27:
        break
    elif key == ord('s'):
        cv2.imwrite('outputs/opticalfb.png', frame2)
        cv2.imwrite('outputs/opticalhsv.png', bgr)
    prvs = next

cap.release()
cv2.waitKey(1)  # This line is not needed when you execute from terminal.
cv2.destroyAllWindows()
cv2.waitKey(1)  # This line is not needed when you execute from terminal.

-1

## Background subtraction
Background subtraction (BS) is a common technique form generating a foreground mask by using static cameras. Only dynamic objects such as people and car should be detected. There are few cases only the background image is obtained. It is more difficult if the shadow is on the image.
### BackgroundSubtractorMOG
This algorithm is based on gaussian mixture distribution. Pixels belonging background are modeled with 3 to 5 mixture distributions.  

In [31]:
cap = cv2.VideoCapture('inputs/vtest.avi')

fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()

while(1):
    ret, frame = cap.read()
    if frame is None:
        break
    
    fgmask = fgbg.apply(frame)
    
    cv2.startWindowThread()  # This line is not needed when you execute from terminal.
    cv2.imshow('frame', fgmask)
    key = cv2.waitKey(30) & 0xff
    if key == 27:
        break
        
cap.release()
cv2.waitKey(1)  # This line is not needed when you execute from terminal.
cv2.destroyAllWindows()
cv2.waitKey(1)  # This line is not needed when you execute from terminal.

-1

### BackgroundSubtractorMOG2
This algorithm is also based on gaussian mixture distribution but it is important in that this algorithm chooses optimal numbers of mixture for each pixel.  
There is an optional argument "detectShadows", which is True by default, whether detect the shadow or not. If set as True, it needs more computational time and shadows are depicted in gray. 

In [32]:
cap = cv2.VideoCapture('inputs/vtest.avi')

fgbg = cv2.createBackgroundSubtractorMOG2()

while(1):
    ret, frame = cap.read()
    if frame is None:
        break
    
    fgmask = fgbg.apply(frame)
    
    cv2.startWindowThread()  # This line is not needed when you execute from terminal.
    cv2.imshow('frame', fgmask)
    key = cv2.waitKey(30) & 0xff
    if key == 27:
        break

cap.release()
cv2.waitKey(1)  # This line is not needed when you execute from terminal.
cv2.destroyAllWindows()
cv2.waitKey(1)  # This line is not needed when you execute from terminal.

-1

### BackgroundSubtractorGMG
This algorithm is combination of statistical background estimation method and region segmentation based on pixel-based Baysian estimation.  
Initial some frames are used to build background model. 

In [33]:
cap = cv2.VideoCapture('inputs/vtest.avi')

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
fgbg = cv2.bgsegm.createBackgroundSubtractorGMG()

while(1):
    ret, frame = cap.read()
    if frame is None:
        break
    
    fgmask = fgbg.apply(frame)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    
    cv2.startWindowThread()  # This line is not needed when you execute from terminal.
    cv2.imshow('frmae', fgmask)
    key = cv2.waitKey(30) & 0xff
    if key == 27:
        break

cap.release()
cv2.waitKey(1)  # This line is not needed when you execute from terminal.
cv2.destroyAllWindows()
cv2.waitKey(1)  # This line is not needed when you execute from terminal.

-1